In [1]:
import pyspark.mllib

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import when  
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

In [3]:
from pyspark.mllib.clustering import KMeans, KMeansModel
import pyspark.mllib
from numpy import array
from math import sqrt
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from pyspark.sql import SQLContext as hv
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

In [4]:
PWD = !pwd
PWD = PWD[0]
from pyspark.sql import SparkSession
app_name = "project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [5]:
!ls project/train_tiny.csv

project/train_tiny.csv


In [6]:
df_tiny = spark.read.csv('project/train_tiny.csv', header = True, inferSchema = True)
rdd_train = sc.textFile('project/train.txt').map(lambda r : r.split('\t'))

In [7]:
df_schema=df_tiny.schema
cols = df_tiny.columns

In [17]:
df=rdd_train.toDF().limit(200).cache()
df.count()

200

In [18]:
df_bkp=df
df.take(1)

[Row(_1='0', _2='1', _3='1', _4='5', _5='0', _6='1382', _7='4', _8='15', _9='2', _10='181', _11='1', _12='2', _13='', _14='2', _15='68fd1e64', _16='80e26c9b', _17='fb936136', _18='7b4723c4', _19='25c83c98', _20='7e0ccccf', _21='de7995b8', _22='1f89b562', _23='a73ee510', _24='a8cd5504', _25='b2cb9c98', _26='37c9c164', _27='2824a5f6', _28='1adce6ef', _29='8ba8b39a', _30='891b62e7', _31='e5ba7672', _32='f54016b9', _33='21ddcdc9', _34='b1252a9d', _35='07b5194c', _36='', _37='3a171ecb', _38='c5c50484', _39='e8b83407', _40='9727dd16')]

In [10]:
#Store the original columns
df_schema=df.schema
cols = df.columns
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
_1,0,0,0,0,0
_2,1,2,2,,3
_3,1,0,0,893,-1
_4,5,44,1,,
_5,0,1,14,,0
_6,1382,102,767,4392,2
_7,4,8,89,,0
_8,15,2,4,0,3
_9,2,2,2,0,0
_10,181,4,245,0,0


In [27]:
#Get numeric and categorical column names
numericCols = df.columns[1:15]
categoricalColumns = df.columns[15:41]

print(numericCols)
print(categoricalColumns)

['_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
['_16', '_17', '_18', '_19', '_20', '_21', '_22', '_23', '_24', '_25', '_26', '_27', '_28', '_29', '_30', '_31', '_32', '_33', '_34', '_35', '_36', '_37', '_38', '_39', '_40']


In [48]:
df=df_bkp
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import udf
#df = df.replace('',None)
def rep_fu(x):
    ret=x
    if x.strip()=='':
        ret=0
    return ret  

def rep_fu_string(x):
    ret=x
    if x.strip()=='':
        ret='__Dummy__'
    return ret

rep_fun_string=udf(rep_fu_string)
df = df.select(*[rep_fun_new(column).alias(column).cast(IntegerType()) if column in numericCols else column for column in df.columns])
df = df.select(*[rep_fun_string(column).alias(column).cast(StringType()) if column in categoricalColumns else column for column in df.columns])
pd.DataFrame(df.take(5),columns=df.columns).transpose()

,0,1,2,3,4
_1,0,0,0,0,0
_2,1,2,2,0,3
_3,1,0,0,893,-1
_4,5,44,1,0,0
_5,0,1,14,0,0
_6,1382,102,767,4392,2
_7,4,8,89,0,0
_8,15,2,4,0,3
_9,2,2,2,0,0
_10,181,4,245,0,0


In [49]:
print(df[df['_1'] == 0].count())
print(df[df['_1'] == 1].count())

156
44


In [50]:
stages = []
indexerCols = []

for categoricalCol in categoricalColumns:
    indexerCol = categoricalCol + "Index"
    indexer = StringIndexer(inputCol=categoricalCol, outputCol= indexerCol).setHandleInvalid("keep")
    stages += [indexer]
    indexerCols.append(indexerCol)

label_stringIdx = StringIndexer(inputCol = '_1', outputCol = 'output')
stages += [label_stringIdx]

In [51]:

assembler = VectorAssembler(inputCols=indexerCols + numericCols, outputCol="features")
stages += [assembler]

In [52]:

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)

selectedCols = ['output', 'features'] + cols
df = df.select(selectedCols)

pd.DataFrame(df.take(5), columns=df.columns).transpose()

Py4JJavaError: An error occurred while calling o6556.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 373.0 failed 1 times, most recent failure: Lost task 0.0 in stage 373.0 (TID 906, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<_16Index:double,_17Index:double,_18Index:double,_19Index:double,_20Index:double,_21Index:double,_22Index:double,_23Index:double,_24Index:double,_25Index:double,_26Index:double,_27Index:double,_28Index:double,_29Index:double,_30Index:double,_31Index:double,_32Index:double,_33Index:double,_34Index:double,_35Index:double,_36Index:double,_37Index:double,_38Index:double,_39Index:double,... 15 more fields>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.ap